# Aula02.Ex08 - Gradiente

### Primeiro, vamos importar as bibliotecas necessárias.
Verifique no código anterior um script para instalar as dependências necessárias (OpenGL e GLFW) antes de prosseguir.

In [106]:
import glfw
from OpenGL.GL import *
import OpenGL.GL.shaders
import numpy as np

### Inicializando janela

In [107]:
glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE)
window = glfw.create_window(800, 800, "Cores", None, None)
glfw.make_context_current(window)

### Capturando eventos de teclado e mouse

In [108]:
R, G, B = 0.5, 0.5, 0.5
x, y = 0, 0
inc = 0.01

keys_pressed: set = set()


def color_update():
    global keys_pressed, R, G, B, inc
    if 263 in keys_pressed:
        R = max(0.0, R - inc)
    if 262 in keys_pressed:
        R = min(1.0, R + inc)
    if 265 in keys_pressed:
        G = max(0.0, G - inc)
    if 264 in keys_pressed:
        G = min(1.0, G + inc)
    if 32 in keys_pressed:
        B = min(1.0, B + inc)
    if 340 in keys_pressed:
        B = max(0.0, B - inc)
    print(R, G, B)
    glUniform4f(loc_color, R, G, B, 1.0)


def position_update():
    global x, y
    # loc is 4 vertices
    # use x and y as starting point for all vertices
    # so if x = 0 and y = 0, the square will start at upper left and end at lower right
    # if x = 1 and y = -1, the square wont be visible
    """
    vertices['position'] = [
        (-1.0, 1.0),  # vertice 0
        (1.0, 1.0),  # vertice 1
        (-1.0, -1.0),  # vertice 3
        (1.0, -1.0),  # vertice 2
    ]
    """
    # move entire square
    if glfw.KEY_A in keys_pressed:
        x -= inc
    if glfw.KEY_D in keys_pressed:
        x += inc
    if glfw.KEY_W in keys_pressed:
        y += inc
    if glfw.KEY_S in keys_pressed:
        y -= inc
    
    # now, to set the new position, we need to update the vertices
    # we can do this by creating a new array with the new positions
    # and then upload it to the GPU
    new_vertices = np.zeros(4, [("position", np.float32, 2)])
    new_vertices['position'] = [
        (-1.0 + x, 1.0 + y),  # vertice 0
        (1.0 + x, 1.0 + y),  # vertice 1
        (-1.0 + x, -1.0 + y),  # vertice 3
        (1.0 + x, -1.0 + y),  # vertice 2
    ]
    glBufferData(GL_ARRAY_BUFFER, new_vertices.nbytes, new_vertices, GL_DYNAMIC_DRAW)
    glBindBuffer(GL_ARRAY_BUFFER, buffer)
    glVertexAttribPointer(loc, 2, GL_FLOAT, False, new_vertices.strides[0], ctypes.c_void_p(0))
    

def key_event(window, key, scancode, action, mods):
    global keys_pressed

    if action == glfw.PRESS:
        keys_pressed.add(key)

    elif action == glfw.RELEASE:
        keys_pressed.discard(key)


glfw.set_key_callback(window, key_event)


def mouse_event(window, button, action, mods):
    pass


glfw.set_mouse_button_callback(window, mouse_event)


### GLSL (OpenGL Shading Language)

Aqui veremos nosso primeiro código GLSL.

É uma linguagem de shading de alto nível baseada na linguagem de programação C.

Nós estamos escrevendo código GLSL como se "strings" de uma variável (mas podemos ler de arquivos texto). Esse código, depois, terá que ser compilado e linkado ao nosso programa. 

Iremos aprender GLSL conforme a necessidade do curso. Usarmos uma versão do GLSL mais antiga, compatível com muitos dispositivos.

### GLSL para Vertex Shader

No Pipeline programável, podemos interagir com Vertex Shaders.

No código abaixo, estamos fazendo o seguinte:

* Definindo uma variável chamada position do tipo vec2.
* Usamos vec2, pois nosso programa (na CPU) irá enviar apenas duas coordenadas para plotar um ponto. Podemos mandar três coordenadas (vec3) e até mesmo quatro coordenadas (vec4).
* void main() é o ponto de entrada do nosso programa (função principal)
* gl_Position é uma variável especial do GLSL. Variáveis que começam com 'gl_' são desse tipo. Nesse caso, determina a posição de um vértice. Observe que todo vértice tem 4 coordenadas, por isso nós combinamos nossa variável vec2 com uma variável vec4.

In [109]:
vertex_code = """

        attribute vec2 position;
        void main(){
            gl_Position = vec4(position,0.0,1.0);
        }
        """

### GLSL para Fragment Shader

No Pipeline programável, podemos interagir com Fragment Shaders.

No código abaixo, estamos fazendo o seguinte:

* void main() é o ponto de entrada do nosso programa (função principal)
* gl_FragColor é uma variável especial do GLSL. Variáveis que começam com 'gl_' são desse tipo. Nesse caso, determina a cor de um fragmento. Nesse caso é um ponto, mas poderia ser outro objeto (ponto, linha, triangulos, etc).

### Possibilitando modificar a cor.

Nos exemplos anteriores, a variável gl_FragColor estava definida de forma fixa (com cor R=0, G=0, B=0).

Agora, nós vamos criar uma variável do tipo "uniform", de quatro posições (vec4), para receber o dado de cor do nosso programa rodando em CPU.

In [110]:
fragment_code = """
        uniform vec4 color;

        void main(){
            vec2 screen_size = vec2(800, 800);
            vec3 cor2 = vec3(0.0, 0.0, 1.0);
            vec2 xy = gl_FragCoord.xy / screen_size;
        
            float mix_value = distance(xy, vec2(0, 1));
            vec3 cor_gradient = mix(color, cor2, mix_value);
        
            // Use the input color as one of the gradient components
            gl_FragColor = vec4(cor_gradient,1.0);
        }
        """

### Requisitando slot para a GPU para nossos programas Vertex e Fragment Shaders

In [111]:
# Request a program and shader slots from GPU
program = glCreateProgram()
vertex = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)


### Associando nosso código-fonte aos slots solicitados

In [112]:
# Set shaders source
glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)

### Compilando o Vertex Shader

Se há algum erro em nosso programa Vertex Shader, nosso app para por aqui.

In [113]:
# Compile shaders
glCompileShader(vertex)
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")


### Compilando o Fragment Shader

Se há algum erro em nosso programa Fragment Shader, nosso app para por aqui.

In [114]:
glCompileShader(fragment)
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Fragment Shader")

### Associando os programas compilado ao programa principal

In [115]:
# Attach shader objects to the program
glAttachShader(program, vertex)
glAttachShader(program, fragment)


### Linkagem do programa

In [116]:
# Build program
glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError('Linking error')

# Make program the default program
glUseProgram(program)

### Preparando dados para enviar a GPU

Nesse momento, nós compilamos nossos Vertex e Program Shaders para que a GPU possa processá-los.

Por outro lado, as informações de vértices geralmente estão na CPU e devem ser transmitidas para a GPU.


In [117]:
# preparando espaço para 3 vértices usando 2 coordenadas (x,y)
vertices = np.zeros(4, [("position", np.float32, 2)])

In [118]:
# preenchendo as coordenadas de cada vértice
vertices['position'] = [
    (-1.0, 1.0),  # vertice 0
    (1.0, 1.0),  # vertice 1
    (-1.0, -1.0),  # vertice 3
    (1.0, -1.0),  # vertice 2
]

### Para enviar nossos dados da CPU para a GPU, precisamos requisitar um slot.

In [119]:
# Request a buffer slot from GPU
buffer = glGenBuffers(1)
# Make this buffer the default one
glBindBuffer(GL_ARRAY_BUFFER, buffer)


### Abaixo, nós enviamos todo o conteúdo da variável vertices.

Veja os parâmetros da função glBufferData [https://www.khronos.org/registry/OpenGL-Refpages/gl4/html/glBufferData.xhtml]

In [120]:
# Upload data
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_DYNAMIC_DRAW)
glBindBuffer(GL_ARRAY_BUFFER, buffer)

### Associando variáveis do programa GLSL (Vertex Shaders) com nossos dados

Primeiro, definimos o byte inicial e o offset dos dados.

In [121]:
# Bind the position attribute
# --------------------------------------
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)


Em seguida, soliciamos à GPU a localização da variável "position" (que guarda coordenadas dos nossos vértices). Nós definimos essa variável no Vertex Shader.

In [122]:
loc = glGetAttribLocation(program, "position")
glEnableVertexAttribArray(loc)

A partir da localização anterior, nós indicamos à GPU onde está o conteúdo (via posições stride/offset) para a variável position (aqui identificada na posição loc).

Outros parâmetros:

* Definimos que possui duas coordenadas
* Que cada coordenada é do tipo float (GL_FLOAT)
* Que não se deve normalizar a coordenada (False)

Mais detalhes: https://www.khronos.org/registry/OpenGL-Refpages/gl4/html/glVertexAttribPointer.xhtml

In [123]:
glVertexAttribPointer(loc, 2, GL_FLOAT, False, stride, offset)

###  Novidade aqui! Vamos pegar a localização da variável color (uniform) para que possamos alterá-la em nosso laço da janela!

In [124]:
loc_color = glGetUniformLocation(program, "color")

### Nesse momento, nós exibimos a janela!


In [125]:
glfw.show_window(window)

### Loop principal da janela.
Enquanto a janela não for fechada, esse laço será executado. É neste espaço que trabalhamos com algumas interações com a OpenGL.

A novidade agora é a função glDrawArrays()

Tal função recebe o tipo de primitiva (GL_TRIANGLES), o índice inicial do array de vértices (vamos mostrar todos os três vértices, por isso começamos com 0) e a quantidade de vértices ( len(vertices) ).

In [126]:
try:
    while not glfw.window_should_close(window):
        glfw.poll_events()
    
        # glPolygonMode(GL_FRONT_AND_BACK,GL_LINE)
        glClear(GL_COLOR_BUFFER_BIT)
        glClearColor(1.0, 1.0, 1.0, 1.0)
    
        glDrawArrays(GL_TRIANGLE_STRIP, 0, 4)
        color_update()
        position_update()
    
        glfw.swap_buffers(window)
except Exception as e:
    print(e)
finally:
    glfw.terminate()

0.5 0.5 0.5
0.5 0.5 0.5
0.5 0.5 0.5
0.5 0.5 0.5
0.5 0.5 0.5
0.5 0.5 0.5
0.5 0.5 0.5
0.5 0.5 0.5
0.5 0.5 0.5
0.5 0.5 0.5
0.5 0.5 0.5
0.5 0.5 0.5
0.5 0.5 0.5
0.5 0.5 0.5
0.5 0.5 0.5
0.5 0.5 0.5
0.5 0.5 0.5
0.5 0.5 0.5
0.5 0.5 0.5
0.5 0.5 0.5
0.5 0.5 0.5
0.5 0.5 0.5
0.5 0.5 0.5
0.5 0.5 0.5
0.5 0.5 0.5
0.5 0.5 0.5
0.5 0.5 0.5
0.5 0.5 0.5
0.5 0.5 0.5
0.5 0.5 0.5
0.5 0.5 0.5
0.5 0.5 0.5
0.5 0.5 0.5
0.5 0.5 0.5
0.5 0.5 0.5
0.5 0.5 0.5
0.5 0.5 0.5
0.5 0.5 0.5
0.5 0.5 0.5
0.5 0.5 0.5
0.5 0.5 0.5
0.5 0.5 0.5
0.5 0.5 0.5
0.5 0.5 0.5
0.5 0.5 0.5
0.5 0.5 0.5
0.5 0.5 0.5
0.5 0.5 0.5
0.5 0.5 0.5
0.5 0.5 0.5
0.5 0.5 0.5
0.5 0.5 0.5
0.5 0.5 0.5
0.5 0.5 0.5
0.5 0.5 0.5
0.5 0.5 0.5
0.5 0.5 0.5
0.5 0.5 0.5
0.5 0.5 0.5
0.5 0.5 0.5
0.5 0.5 0.5
0.5 0.5 0.5
0.5 0.5 0.5
0.5 0.5 0.5
0.5 0.5 0.5
0.5 0.5 0.5
0.5 0.5 0.5
0.5 0.5 0.5
0.5 0.5 0.5
0.5 0.5 0.5
0.5 0.5 0.5
0.5 0.5 0.5
0.5 0.5 0.5
0.5 0.5 0.5
0.5 0.5 0.5
0.5 0.5 0.5
0.5 0.5 0.5
0.5 0.5 0.5
0.5 0.5 0.5
0.5 0.5 0.5
0.5 0.5 0.5
0.5 0.5 0.5
0.5 0.5 0.5
0.5 

# Exercício

Modifique esse código para variar a cor do objeto dentro do loop principal da janela.
